In [ ]:
!pip install pyspark
!wget https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/posts_sample.xml
!wget https://git.ai.ssau.ru/tk/big_data/raw/branch/master/data/programming-languages.csv

In [12]:
import pyspark
import os

from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql import SparkSession

In [13]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.13.0 pyspark-shell'
SparkS = SparkSession.builder.appName("l2").master("local[*]").getOrCreate()

prog_lang_list = [str(x[0]) for x in SparkS.read.csv("programming-languages.csv").collect()]

posts = SparkS.read.format("xml").options(rowTag="row").load('posts_sample.xml')

In [14]:
def language_detection(x):
  tag = None
  for language in prog_lang_list:
    if "<" + language.lower() + ">" in x._Tags.lower():
      tag = language
      break
  if tag is None:
    return None
  return (x._Id, tag)


def check_date(x, year):
  return x._CreationDate >= datetime(year=year, month=1, day=1) and x._CreationDate <= datetime(year=year, month=12, day=31)

In [15]:
res = {}

for year in range(2010, 2020):
  res[year] = posts.rdd\
      .filter(lambda x: x._Tags is not None and check_date(x, year))\
      .map(language_detection).filter(lambda x: x is not None)\
      .keyBy(lambda x: x[1])\
      .aggregateByKey(0, lambda x, y: x + 1, lambda x1, x2: x1 + x2,)\
      .sortBy(lambda x: x[1], ascending=False).toDF()

  res[year] = res[year].select(col("_1").alias("Язык программирования"),
                               col("_2").alias(f"Количество упоминаний в {year}")).limit(10)
  res[year].write.format("parquet").save(f"{year} год")
  res[year].show()

+---------------------+----------------------------+
|Язык программирования|Количество упоминаний в 2010|
+---------------------+----------------------------+
|                 Java|                          52|
|           JavaScript|                          44|
|                  PHP|                          42|
|               Python|                          25|
|          Objective-C|                          22|
|                    C|                          20|
|                 Ruby|                          11|
|               Delphi|                           7|
|                    R|                           3|
|                 Bash|                           3|
+---------------------+----------------------------+

+---------------------+----------------------------+
|Язык программирования|Количество упоминаний в 2011|
+---------------------+----------------------------+
|                  PHP|                          97|
|                 Java|                      

In [ ]:
!zip -r '2010 год' '2010 год'
!zip -r '2011 год' '2011 год'
!zip -r '2012 год' '2012 год'
!zip -r '2013 год' '2013 год'
!zip -r '2014 год' '2014 год'
!zip -r '2015 год' '2015 год'
!zip -r '2016 год' '2016 год'
!zip -r '2017 год' '2017 год'
!zip -r '2018 год' '2018 год'
!zip -r '2019 год' '2019 год'